In [3]:
import pandas as pd
import datetime
import json
import numpy as np
import string
import math
import re
import matplotlib.pyplot as plt
import seaborn as sns

from torch.utils.data import TensorDataset

import torchtext
from torchtext.vocab import Vectors
from torchtext import data, datasets

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# from utils.func import huber, tokenizer_with_preprocessing, norm_time


In [7]:
for i, date in enumerate(range(2011, 2020)):
    tmp = pd.read_csv('../data/news/' + str(date) + '.csv', encoding='cp932')
#     tmp = tmp[tmp['Company_IDs(TSE)'].isin(code_list)]
    tmp = tmp[['Time_Stamp_Original(JST)', 
                       'Company_Code(TSE)', 
                       'Headline', 
                       'News_Source',
                       'Company_Relevance', 
                       'Keyword_Article']]

    # 欠損除去
    tmp = tmp[~tmp["Keyword_Article"].isnull()]

    # タグ除去
    tmp = tmp[(tmp['News_Source'] == '日経') | 
                    (tmp['News_Source'] == 'ＮＱＮ') |
                    (tmp['News_Source'] == 'ＱＵＩＣＫ') | 
                    (tmp['News_Source'] == 'Ｒ＆Ｉ')]

#     tmp['code'] = tmp['Company_Code(TSE)'].astype(int)
#     tmp['date'] = pd.to_datetime(tmp["Time_Stamp_Original(JST)"]).map(norm_time)
#     tmp = tmp.set_index(['date', 'code'], drop=True)
#     tmp = tmp.drop(['Time_Stamp_Original(JST)', 'Company_Code(TSE)'], axis=1)

    if i == 0:
        df1 = tmp.copy()
    else:
        df1 = pd.concat([df1, tmp])

In [16]:
df1['Company_Code(TSE)'].value_counts().index[:20]

Float64Index([8301.0, 7203.0, 9501.0, 9984.0, 6502.0, 8306.0, 6758.0, 7201.0,
              8411.0, 6501.0, 6752.0, 9983.0, 6753.0, 8604.0, 7267.0, 8316.0,
              5401.0, 6702.0, 9503.0, 7974.0],
             dtype='float64')

In [239]:
code_list = ['7203', '8301', '9501']

In [240]:
for i, date in enumerate(range(2011, 2013)):
    tmp = pd.read_csv('./data/news/' + str(date) + '.csv', encoding='cp932')
    tmp = tmp[tmp['Company_IDs(TSE)'].isin(code_list)]
    tmp = tmp[['Time_Stamp_Original(JST)', 
                       'Company_Code(TSE)', 
                       'Headline', 
                       'News_Source',
                       'Company_Relevance', 
                       'Keyword_Article']]

    # 欠損除去
    tmp = tmp[~tmp["Keyword_Article"].isnull()]

    # タグ除去
    tmp = tmp[(tmp['News_Source'] == '日経') | 
                    (tmp['News_Source'] == 'ＮＱＮ') |
                    (tmp['News_Source'] == 'ＱＵＩＣＫ') | 
                    (tmp['News_Source'] == 'Ｒ＆Ｉ')]

    tmp['code'] = tmp['Company_Code(TSE)'].astype(int)
    tmp['date'] = pd.to_datetime(tmp["Time_Stamp_Original(JST)"]).map(norm_time)
    tmp = tmp.set_index(['date', 'code'], drop=True)
    tmp = tmp.drop(['Time_Stamp_Original(JST)', 'Company_Code(TSE)'], axis=1)

    if i == 0:
        df1 = tmp.copy()
    else:
        df1 = pd.concat([df1, tmp])

In [203]:
# 株価を取り出す
for i, code in enumerate(code_list):
    tmp = pd.read_csv('./data/stock_price/' + str(code) + '.csv', index_col=0)
    tmp['code'] = code
    if i == 0:
        df2 = tmp
    else:
        df2 = pd.concat([df2, tmp])

df2['date'] = pd.to_datetime(df2['date'])
df2['code'] = df2['code'].astype(int)
df2 = df2.set_index(['date', 'code'], drop=True)

In [204]:
df3.head()

Headline News_Source  \
date       code                                                      
2011-01-04 8301    <NQN>☆今週の短期　年末通過で資金余剰感強まる　翌日物は低位安定か         ＮＱＮ   
2011-01-05 7203               <日経>◇次世代車の研究開発　名大に国内最大拠点          日経   
2011-01-06 8301  <NQN>☆11年金融政策の焦点(1)デフレの議論混乱を懸念・福田東大教授         ＮＱＮ   
           7203                      <日経>◇日本経済新聞５日夕刊早版          日経   
2011-01-07 7203   <NQN>◇トヨタ社長「今年は後半に晴れ間」　為替は１ドル＝90円を期待         ＮＱＮ   

                Company_Relevance  \
date       code                     
2011-01-04 8301                47   
2011-01-05 7203                38   
2011-01-06 8301                49   
           7203                10   
2011-01-07 7203               100   

                                                   Keyword_Article  adj_close  \
date       code                                                                 
2011-01-04 8301  通過:余剰:水準:安定:推移:資金供給:１２月:連続:維持:１月:ペース:通常:速い:可能性...    53300.0   
2011-01-05 7203  安全:環境:負荷:開発:目指す:開所式:研究拠点:効率:簡素化:次世代:電気自動車:電気:幅...     3295.0   
2011-01-06 8301  外国為替市場:円相場:上昇:全体:先行き:不透明:包括:緩和:導入:関心:政策:調査:部長:...    53300.0   
           7203  加速:環境:増:後退:貢献:債券:投資:温暖化対策:ワクチン:マグロ:初セリ:円相場:反発:...     3380.0   
2011-01-07 7203  豊田:見通し:販売:エコカー補助金:安定的:伸び:株価:為替:水準:日経平均株価:最低:ライ...     3455.0   

                    price  
date       code            
2011-01-04 8301       NaN  
2011-01-05 7203  0.000000  
2011-01-06 8301  2.579666  
           7203  2.218935  
2011-01-07 7203  1.302460

In [205]:
df3['adj_close'].groupby(level=['code']).diff().shift(-1)

date        code
2011-01-04  8301       NaN
2011-01-05  7203       0.0
2011-01-06  8301      85.0
            7203      75.0
2011-01-07  7203      45.0
                     ...  
2012-12-26  7203    1600.0
2012-12-27  8301     100.0
            7203    -100.0
2012-12-28  8301      75.0
            7203       NaN
Name: adj_close, Length: 766, dtype: float64

,,Headline,News_Source,Company_Relevance,Keyword_Article,adj_close,price
date,code,,,,,,
2011-01-05,7203,<日経>◇次世代車の研究開発 名大に国内最大拠点,日経,38,安全:環境:負荷:開発:目指す:開所式:研究拠点:効率:簡素化:次世代:電気自動車:電気:幅...,3295.0,NaN
2011-01-06,7203,<日経>◇日本経済新聞５日夕刊早版,日経,10,加速:環境:増:後退:貢献:債券:投資:温暖化対策:ワクチン:マグロ:初セリ:円相場:反発:...,3380.0,0.050327
2011-01-07,7203,<NQN>◇トヨタ社長「今年は後半に晴れ間」 為替は１ドル＝90円を期待,ＮＱＮ,100,豊田:見通し:販売:エコカー補助金:安定的:伸び:株価:為替:水準:日経平均株価:最低:ライ...,3455.0,1.302460
2011-01-11,7203,<日経>◇ＧＭ、ビュイック初の「高級小型車」を発表,日経,36,開幕:北米:ブランド:小型車:新型車:高級車:高い:高級ブランド:<pad>:開幕:位置付け...,3455.0,2.315485
2011-01-12,7203,<日経>◇トヨタ、米ミシガン州に安全研究センター新設,日経,100,先進:安全:子供:高齢者:事故:向上:目指す:米国:大規模:リコール:回収:問題:開催:豊田...,3500.0,-1.285714
...,...,...,...,...,...,...,...
2012-12-21,9501,<日経>◇「震災関連死」初の和解 東電が賠償1460万円,日経,100,事故:死亡:女性:政府:申し立て:和解:震災:原発:病院:４月:移動:内容,224.0,1.747312
2012-12-25,9501,<日経>◇全国紙朝刊トップニュース（東京）・25日,日経,46,原発:東京:新聞:届か:ず:送付:拡大:組織的:休職:補修:債務:完済:延期:<pad>:停...,229.0,1.188904
2012-12-26,9501,<日経>◇東電、7000億円追加申請へ 累計で３兆円超に,日経,100,事故:必要:追加:申請:方針:土地:上回る:判断:援助:１１月:当初:想定:膨らむ:範囲:可...,228.0,-3.508772


In [232]:
# 時系列をくっつける
df3 = pd.concat([df1,df2], axis=1, join_axes=[df1.index], levels=[0,1])
df3 = df3.sort_values(by=['code', 'date'])
df3['Keyword_Article'] = \
  df3.groupby(level=[0,1]).apply(lambda x: ':<pad>:'.join(list(x['Keyword_Article'])))

df3 = df3.dropna()
df3 = df3[~df3.duplicated(subset=['Keyword_Article'])]
df3['price'] = \
        df3['adj_close'].groupby(level=['code']).pct_change(1).shift(-1)*100
# df3 = df3.dropna()
df3.head()

/home/ts-zemi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  


,,Headline,News_Source,Company_Relevance,Keyword_Article,adj_close,price
date,code,,,,,,
2011-01-05,7203,<日経>◇次世代車の研究開発 名大に国内最大拠点,日経,38,安全:環境:負荷:開発:目指す:開所式:研究拠点:効率:簡素化:次世代:電気自動車:電気:幅...,3295.0,2.579666
2011-01-06,7203,<日経>◇日本経済新聞５日夕刊早版,日経,10,加速:環境:増:後退:貢献:債券:投資:温暖化対策:ワクチン:マグロ:初セリ:円相場:反発:...,3380.0,2.218935
2011-01-07,7203,<NQN>◇トヨタ社長「今年は後半に晴れ間」 為替は１ドル＝90円を期待,ＮＱＮ,100,豊田:見通し:販売:エコカー補助金:安定的:伸び:株価:為替:水準:日経平均株価:最低:ライ...,3455.0,0.000000
2011-01-11,7203,<日経>◇ＧＭ、ビュイック初の「高級小型車」を発表,日経,36,開幕:北米:ブランド:小型車:新型車:高級車:高い:高級ブランド:<pad>:開幕:位置付け...,3455.0,1.302460
2011-01-12,7203,<日経>◇トヨタ、米ミシガン州に安全研究センター新設,日経,100,先進:安全:子供:高齢者:事故:向上:目指す:米国:大規模:リコール:回収:問題:開催:豊田...,3500.0,1.000000


In [207]:
# CSVファイルに保存する
df4 = pd.concat([df3[['Keyword_Article', 'price']].rename(
                               columns={'Keyword_Article': 'state', 'price': 'reward'}),
                         df3[['Keyword_Article']].shift(-1).rename(
                               columns={'Keyword_Article': 'next_state'})], axis=1).dropna()
df4 = df4[['state', 'next_state', 'reward']]

date_year = df4.index.map(lambda x: x[0].year)

In [234]:
df4.reward

-27.6223776224

# Dataの作成

In [209]:
max_length = 256
batch_size = 32

# 読み込んだ内容に対して行う処理を定義
TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing, 
                            use_vocab=True,
                            lower=True, include_lengths=True, batch_first=True, fix_length=max_length, 
                            init_token="<cls>", eos_token="<eos>")
LABEL = torchtext.data.Field(sequential=False, use_vocab=False, dtype=torch.float)

In [210]:
train_ds = torchtext.data.TabularDataset.splits(
    path='./data/news', train='text_train.tsv',
    format='tsv',
    fields=[('Text1', TEXT), ('Text2', TEXT), ('Label', LABEL)])
train_ds = train_ds[0]

test_ds = torchtext.data.TabularDataset.splits(
    path='./data/news', train='text_test.tsv',
    format='tsv',
    fields=[('Text1', TEXT), ('Text2', TEXT), ('Label', LABEL)])
test_ds = test_ds[0]

# japanese_fasttext_vectors = Vectors(name='./data/news/cc.ja.300.vec')
TEXT.build_vocab(train_ds, 
#                  vectors=japanese_fasttext_vectors,
                 min_freq=10)
TEXT.vocab.freqs

train_dl = torchtext.data.Iterator(
    train_ds, batch_size=batch_size, train=True)
test_dl = torchtext.data.Iterator(
    test_ds, batch_size=100, train=True, shuffle=False)

In [ ]:
batch = next(iter(train_dl))
print(batch.Text1)
print(batch.Text2)
print(batch.Label)

# モデル構築

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
batch.Text1[0].shape

In [ ]:
class IQN(nn.Module):
    def __init__(self, text_embedding_vector, vocab_size, embedding_dim,
                 n_filters, filter_sizes, pad_idx,
                 d_model=300, n_actions=1, n_quant=8):
        super().__init__()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        self.n_actions = n_actions
        self.n_quant = n_quant
        
        self.embedding = nn.Embedding(
            vocab_size, 
            embedding_dim, 
            padding_idx=pad_idx)


        self.phi = nn.Linear(64, 64)
        # self.phi_bias = nn.Parameter(torch.zeros(64))

        # self.fc0 = nn.Linear(n_state, 64)
        self.convs = nn.ModuleList([
            nn.Conv2d(in_channels=1,
                      out_channels=n_filters,
                      kernel_size=(fs, embedding_dim))
            for fs in filter_sizes
        ])

        self.fc0 = nn.Linear(len(filter_sizes) * n_filters, 64)

        self.fc1 = nn.Linear(64, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc_q = nn.Linear(64, n_actions)

    def forward(self, text, eta=1.0):
        # [batch_size, num_state]
        mb_size = text.size(0)

        # [batch_size, sen_len] → [batch_size, sen_len, emb_dim]
        embedded = self.embedding(text)
        
        # [batch size, 1, sen len, emb dim]
        embedded = embedded.unsqueeze(1)

        # [batch size, n_filters, sen len - filter_sizes[n] + 1]
        x = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]

        # [batch size, n_filters, sen len - filter_sizes[n] + 1]
        x = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in x]

        # [batch size, n_filters * len(filter_sizes)]
        x = torch.cat(x, dim=1)

        # [batch size, 64]
        x = F.relu(self.fc0(x))

        # [batch_size, 64] 
        # → [batch_size, quant, 64]
        x = x.repeat(1, 1, self.n_quant).view(mb_size, self.n_quant, 64)

        # [batch_size, quant, 1]
        tau = eta * torch.rand(mb_size, self.n_quant).to(self.device).view(mb_size, -1, 1)

        # [1, 64]
        pi_mtx = math.pi * torch.arange(0, 64, 1.0).to(self.device).unsqueeze(0)

        # [batch_size, quant, 64]
        cos_tau = torch.cos(torch.matmul(tau, pi_mtx))

        # [batch_size, quant, quantile_embedding_di64]
        phi = F.relu(self.phi(cos_tau))

        # [batch_size, quant, 4]
        h = x * phi

        # [batch_size, quant, 64]
        h = F.relu(self.fc1(h))
        h = F.relu(self.fc2(h))

        # [batch_size, quant, 2]
        action_value = self.fc_q(h).view(mb_size, self.n_quant, self.n_actions)
        # action_value = action_value.transpose(0, 2, 1)

        return action_value, tau

In [ ]:
VOCAB_SIZE = len(TEXT.vocab.freqs)
EMBEDDING_DIM = 300
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
PAD_IDX = 1
N_QUANT = 64
GAMMA = 0.99

In [ ]:
model = IQN(TEXT.vocab.vectors, VOCAB_SIZE, EMBEDDING_DIM, N_FILTERS,
                        FILTER_SIZES, PAD_IDX, n_quant=N_QUANT)

target_model = IQN(TEXT.vocab.vectors, VOCAB_SIZE, EMBEDDING_DIM, N_FILTERS,
                        FILTER_SIZES, PAD_IDX, n_quant=N_QUANT)

model = model.to(device)
target_model = target_model.to(device)

target_model.load_state_dict(model.state_dict())

# 最適化

In [ ]:
# 最適化手法
learning_rate = 2.5e-5
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
num_epochs = 30
TARGET_UPDATE_FREQ = 10
# dataloaders_dict = {'train': train_dl, 'val':val_dl}
dataloaders_dict = {'train': train_dl}

print('----start----')

torch.backends.cudnn.benchmark = True

for epoch in range(num_epochs):
    epi_rewards = []
    
    # update target_model
    if epoch % TARGET_UPDATE_FREQ == 0:
        target_model.load_state_dict(model.state_dict())
    
    for batch in (dataloaders_dict['train']):      
        # curr_q
        states = batch.Text1[0].to(device)
        next_states = batch.Text2[0].to(device)
        rewards = batch.Label.to(device)
    
        curr_q, tau = model(states)
        action_value = curr_q.mean(dim=1)
        
        curr_q = curr_q.repeat(1, 1, N_QUANT)
        
        action_value[action_value > 0] = 1
        action_value[action_value <= 0] = 0
        selected_actions = action_value.cpu().detach().numpy().reshape(-1)
        
        epi_rewards.append((
            selected_actions * rewards.detach().cpu().numpy()).sum())
        

        # target_q
        with torch.no_grad():
            next_q, _ = target_model(next_states)
            next_q = next_q.squeeze(2)

            target_q = rewards.reshape(-1, 1) + GAMMA * next_q
            target_q = target_q.unsqueeze(2)
            target_q = target_q.repeat(1, 1, N_QUANT)
            target_q = target_q.permute(0, 2, 1)

        # (BATCH, N_QUANT, N_QUANT)
        tau = tau.repeat(1, 1, N_QUANT)
        diff = target_q - curr_q

        loss = huber(diff)

        I_delta = (diff<0).double()
        loss *= torch.abs(tau - I_delta)

        # huber loss
        loss = torch.mean(torch.sum(torch.mean(loss, dim=2), dim=1))

        # backprop loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print('--------------------')
    print('epoch:', epoch)
    print('loss:', loss.item())
    print('epi_reward:', sum(epi_rewards))

In [ ]:
(target_q - curr_q).shape

In [ ]:
curr_q.shape

# 描画

In [ ]:
batch = next(iter(train_dl))
states = batch.Text1[0].to(device)
next_states = batch.Text2[0].to(device)
rewards = batch.Label.to(device)

In [ ]:
batch = next(iter(test_dl))
print(batch.Text1[0].shape)

In [ ]:
start = 0
N = 10
win = 0
fig, ax = plt.subplots((N-start), 2, figsize=(7, (N-start)*2));
for i in range(start, N):
    states = batch.Text1[0].to(device)
    rewards = batch.Label.to(device)
    curr_q, _ = model(states[i].unsqueeze(0), eta=1)
    dist_hist = curr_q.view(-1).cpu().detach().numpy()
    sns.distplot(dist_hist, bins=51, color='red', ax=ax[i - start][0], label=str(rewards[i].item()))
    
    cvar, _ = model(states[i].unsqueeze(0), eta=0.1)
    cvar_dist = cvar.view(-1).cpu().detach().numpy()
    sns.distplot(cvar_dist, bins=51, color='blue', ax=ax[i - start][1], label=str(rewards[i].item()))
    
    # 期待値
    ax[i - start][0].vlines(
        np.round(dist_hist.mean(), 3), 
        ymin=0, ymax=0.3, color='red', label=str(curr_q.mean().item()))

    print('i = ', i,
              '実際: ', rewards[i].cpu().detach().numpy(),
              '平均: ', curr_q.mean().item(),
              'SR: ', np.round(dist_hist.mean() / dist_hist.std(), 3))

    print('CVaR 平均: ', np.round(cvar_dist.mean(), 3),
              'CVaR SR: ', np.round(cvar_dist.mean() / cvar_dist.std(), 3))

    ax[i-start][0].legend()

In [ ]:
win / (N-start)